Урок по использованию Google Coral AI Accelerator Module: тренируем модель для распознавания картинок подводным аппаратом.

# Обучаем модель

In [ ]:
'''
Перед тем как начать, нужно сначала загрузить датасет.
Нижеуказанный архив содержит размеченные изображения,
разбитые на три группы (train, validation, test).

Загрузка и распаковка архива:
'''

!wget "https://raw.githubusercontent.com/murproject/lessons/master/18.%20Objects%20detection%20with%20Google%20Coral/dataset.zip"
!unzip -o dataset.zip

In [ ]:
'''
Установим необходимые библиотеки из репозитория PIP.
'''

!pip install -q tflite-model-maker

In [ ]:
'''
Импортируем необходимые нам библиотеки.
'''

import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

In [ ]:
'''
Задаём датасет, состоящий из трёх групп. Указываем путь к распакованным данным,
а затем заполняем массивы данных и выводим количество кадров в каждой группе.
'''

path = '/content/data/'

dataset = {}

for group in ['train', 'validation', 'test']:
    dataset[group] = object_detector.DataLoader.from_pascal_voc(
        label_map = path + 'img_labels.yaml',
        images_dir = path + group,
        annotations_dir = path + group
    )

    print('size of %s:' % group , len(dataset[group]))

In [ ]:
'''
Указываем спецификацию модели, которую будем использовать.
Мы воспользуемся моделью из семейства EfficientDet:
это как раз подходит под наши цели (распознавание объектов),
а также совместимо с Edge TPU (то есть, способно работать
на Coral, т.к. в его основе лежит именно эта технология).

Данное семейство моделей включает несколько вариаций,
отличающихся размером (от модели Lite0 размером 4.4 MB,
до Lite4 размером 19.9 MB), при этом один модуль Coral
не способен работать с моделями более 8 MB. Чем больше
размер модели, тем выше точность, но и выше задержка
распознавания. Мы воспользуемся EfficientDet-Lite2
размером в 7.2 MB - самая объёмная, которая может быть
запущена на одном модуле Coral.
'''

spec = object_detector.EfficientDetLite2Spec()

In [ ]:
'''
Начинаем процесс обучения. Здесь мы указываем переменные с датасетом,
ранее выбраную спецификацию модели, а также ещё важные параметры:
количство эпох и размер одного этапа.

В процессе обучения, происходит многократная тренировка нейронной сети
на всём датасете. Один такой полный проход - эпоха. Если эпох слишком мало,
то сеть может быть недостаточно обучена, а при слишком большом их количестве
возможно переобучение (когда нейросеть, образно говоря, слишком "хорошо запомнила"
обучающую выборку, и плохо срабатывает на данных, которые даже немного от
неё отличаются).

Обучение является довольно затратным процессом, и если попытаться провести
тренировку на всём датасете в один проход, то можно столкнуться, к примеру,
с исчерпанием оперативной памяти. Чтобы этого избежать, каждую эпоху можно разбить
на несколько этапов, чтобы тренировка происходила небольшими порциями.

Оптимальный подбор этих параметров может зависеть от конкретной используемой
модели, а также от доступных ресурсов.

Когда будет происходить процесс обучения, вы сможете наблюдать за ходом выполнения
каждой эпохи: со временем значение ошибки становится всё меньше.
'''

model = object_detector.create(train_data=dataset['train'],
                               model_spec=spec, 
                               validation_data=dataset['validation'], 
                               epochs=50, 
                               batch_size=20, # всего 380 кадров / 20 кадров на этап = 19 этапов
                               train_whole_model=True)

In [ ]:
'''
После того, как модель была обучена, мы можем проверить качество распознавания
на тестовой выборке. Мы можем получить примерный процент точности работы модели.
'''

model.evaluate(dataset['test'], batch_size=4)

In [ ]:
'''
Модель обучена и её работа проверена, и теперь мы можем экспортировать её
в файл. После этого, данную модель уже можно использовать на CPU и GPU
при помощи библиотеки Tensorflow Lite, но на данный момент она ещё непригодна
для использования на Google Coral.
'''

model.export(export_dir='.',
             tflite_filename='rsub_model.tflite',
             label_filename='rsub_labels.txt',
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL])

# Экспортируем модель для Google Coral

In [ ]:
'''
Архитектура модуля Google Coral довольно специфична. Для того, чтобы модель
могла работать на данном оборудовании, необходимо выполнить её преобразование
в формат, совместимый с Edge TPU.

Для начала установим необходимый для этого софт.
'''


! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler

In [ ]:
'''
После установки, можно выполнить преобразование модели.
Выполнение модели может быть распределено на несколько модулей,
но он у нас всего один, что и укажем в соответствующей переменной.
'''

!edgetpu_compiler rsub_model.tflite --num_segments=1

In [ ]:
'''
Всё готово. Теперь мы можем загрузить все результаты нашей работы.
'''

from google.colab import files

files.download('rsub_model.tflite') # модель для запуска с использованием Tensorflow Lite
files.download('rsub_model_edgetpu.tflite') # модель для Edge TPU (Coral)
files.download('rsub_labels.txt') # список категорий (классы объектов)

# Демонстрация работы

Мы можем запустить обученную модель и без модуля Coral, воспользовавшись моделью, которая ещё не была преобразована для Edge TPU.

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

In [ ]:
import os
import cv2
from google.colab.patches import cv2_imshow

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

In [ ]:
path_model = 'rsub_model.tflite'
path_labels = 'rsub_labels.txt'

threshold = 0.3
max_detections = 4

interpreter = tflite.Interpreter(path_model)
interpreter.allocate_tensors()
labels = read_label_file(path_labels)
inference_size = common.input_size(interpreter)

In [ ]:
def process_img(path):
    img = cv2.imread(path)
    img = cv2.resize(img, inference_size)

    common.set_input(interpreter, cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    interpreter.invoke()
    objs = detect.get_objects(interpreter, threshold)[:max_detections]

    drawing = img.copy()

    height, width, channels = img.shape
    scale_x, scale_y = width / inference_size[0], height / inference_size[1]

    print(path, "objects:")
    for obj in objs:
        box = obj.bbox.scale(scale_x, scale_y)
        p0 = (int(box.xmin), int(box.ymin))
        p1 = (int(box.xmax), int(box.ymax))

        percent = int(100 * obj.score)
        label = '{}% {}'.format(percent, labels[obj.id])
        print(label)
        font = cv2.FONT_HERSHEY_DUPLEX

        cv2.rectangle(drawing, p0, p1, (0, 255, 0), 1)
        cv2.putText(drawing, label, (p0[0], p0[1] + 20), font, 0.7, (255, 255, 255), 1)

    cv2_imshow(drawing)
    print()

In [ ]:
dir = "/content/data/test/"

for f in os.listdir(dir):
    if f.endswith(".jpg"):
      process_img(os.path.join(dir, f))